# <center> VIRTUAL PAINT IN AIR CANVAS </center>

Make sure you have read the README.md file. Ensure that you have opened the notebook in Jupyter notebook (Anaconda Navigator)

Click on **Cell > Run All** option to run the notebook (For Anaconda). For closing the window, press 'q' key. For clearing your canvas, select the paint icon in the top left corner of the screen.


**Done by:** Raahul R (Nickname in MM Discord server: Not so shy guy)

# SOURCE CODE:

Importing the required modules

In [1]:
import cv2
import numpy as np
import time
import os
import mediapipe as mp
import HandTrackingModule as htm

The position IDs of the hand is kept as a reference

<img src = "https://cdn.discordapp.com/attachments/1033607158119530587/1092458818287435806/image.png" height = 200px>

The paint frames are located inside the Header folder. It is accessed using the os module.

In [2]:
folderPath = "Header"
myList = os.listdir(folderPath) # List containing the header filenames
#print(myList)
overlay = [] # All the images will be appended here
drawColor = (255, 0, 255) # Setting up default color which will be then changed

Images are imported and stored in a overlay list for overlaying along with webcam

In [3]:
for imPath in myList:
    image = cv2.imread(f'{folderPath}/{imPath}')
    overlay.append(image)

Switching on web cam and setting default head as 1.png. Detector object of class handDetector under the custom module, HandTrackingModule is created.

In [4]:
header = overlay[0]
brushThickness = 15 # Thickness of the brush
eraserThickness = 50 # Thickness of the eraser
cap = cv2.VideoCapture(0)
# Setting the image dimensions as 1280 x 720 
cap.set(3, 1280)
cap.set(4, 720)
imgCanvas = np.zeros((720, 1280, 3), np.uint8) # Setting up the canvas
detector = htm.handDetector() # Our hand detector object from the module
xp, yp = 0, 0 # Brush's previous position

The main loop

In [ ]:
while True:
    # 1. Import image
    success, img = cap.read()
    img = cv2.resize(img, (1280, 720)) # Webcam scaling
    img = cv2.flip(img, 1) # Flip image

    # 2. Find Hand Landmarks
    img = detector.findHands(img)
    lmList = detector.findPosition(img, draw = False)

    if len(lmList) != 0:
        #print(lmList) # testing purpose
        
        x1, y1 = lmList[8][1:] # 8 = tip of index finger
        x2, y2 = lmList[12][1:] # 12 = tip of middle finger

    # 3. Check which fingers are up
    fingers = detector.fingersUp(lmList)
    #print(fingers) # testing purpose

    # 4. If Selection mode - Two fingers are up
    if fingers[1] == 1 and fingers[2] == 1: #Index and middle are up?
        xp, yp = 0, 0 # Setting previous and current zero so that drawing can resume
        # We draw rectangle for selection mode
        # print("Selection Mode") # testing purpose

        if y1 < 125: #In header
            if 250 < x1 < 450: # Picking Pink
                header = overlay[0]
                drawColor = (255, 0, 255) # This is a RGB Tuple

            elif 550 < x1 < 750: # Picking Blue
                header = overlay[1] 
                drawColor = (255, 0, 0)

            elif 800 < x1 < 950: # Picking Green
                header = overlay[2]
                drawColor = (0, 255, 0)

            elif 1050 < x1 < 1200: # Picking Eraser
                header = overlay[3]
                drawColor = (0, 0, 0)

            elif 10 < x1 < 150: # Clear screen by clicking paint icon
                header = overlay[3]
                drawColor = (0, 0, 0)
                imgCanvas = np.zeros((720, 1280, 3), np.uint8)

                
        cv2.rectangle(img, (x1, y1 - 25), (x2, y2 + 25), drawColor, cv2.FILLED)                

    # 5. If Drawing mode - Index finger is up
    if fingers[1] == 1 and fingers[2] == 0:
        cv2.circle(img, (x1, y1), 15, (255, 0, 255), cv2.FILLED)
        # print("Drawing Mode") # testing purpose

        # Drawing concept: We are going to draw little lines, that links both
        # current coordinates as well as previous coordinates
        if xp == 0 and yp == 0:
            xp, yp = x1, y1 # Making (0, 0) the first point as current point for first drawing
        
        # Implementing thicker eraser
        if drawColor == (0, 0, 0):
            cv2.line(img, (xp, yp), (x1, y1), drawColor, eraserThickness)
            cv2.line(imgCanvas, (xp, yp), (x1, y1), drawColor, eraserThickness)
        else:
            cv2.line(img, (xp, yp), (x1, y1), drawColor, brushThickness)
            cv2.line(imgCanvas, (xp, yp), (x1, y1), drawColor, brushThickness)
        xp, yp = x1, y1 # Updating previous coordinates


    # Series of image conversions and logical operators that can merge images together 
    imgGray = cv2.cvtColor(imgCanvas, cv2.COLOR_BGR2GRAY) # Convert to img to gray
    _, imgInv = cv2.threshold(imgGray, 50, 255, cv2.THRESH_BINARY_INV) # Convert gray to binary
    imgInv = cv2.cvtColor(imgInv, cv2.COLOR_GRAY2BGR) # Invert canvas
    img = cv2.bitwise_and(img, imgInv) # And
    img = cv2.bitwise_or(img, imgCanvas) # Or

    #Setting up the header
    img[0: 125, 0: 1280] = header

    # img.addWeighted(img, 0.5, imgCanvas, 0.5) # previous merge idea. But had an issue.
    cv2.imshow("Virtual Painter", img)
    # cv2.imshow("Canvas", imgCanvas) # testing purpose
    
    # Exit if 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()